In [1]:
import numpy as np
import cvxpy as cp

### SunCo production planning

In [3]:
x = cp.Variable(1) # barrels of crude purchased
yf = cp.Variable(1) # barrels of fuel processed in cracker
zf = cp.Variable(1) # barrels of fuel NOT processed in cracker
yh = cp.Variable(1) # barrels of oil processed in cracker
zh = cp.Variable(1) # barrels of oil NOT processed in cracker

In [4]:
constraints = []
constraints += [x <= 20] # upper bound on crude purchase
constraints += [yf + 0.75*yh <= 8] # cracker time constraints
constraints += [yf + zf == 0.5*x] # crude -> fuel
constraints += [yh + zh == 0.5*x] # crude -> heating
constraints += [x >= 0, yf >=0, zf >= 0, yh >= 0, zh >=0]

In [6]:
obj = cp.Maximize(60*zf + 40*zh + 130*yf + 90*yh - 40*x)

In [7]:
prob = cp.Problem(obj,constraints)

In [8]:
prob.solve(solver=cp.GLPK)
if (prob.status == 'optimal'):
    print('optimal value = ' + str(prob.value))
    print('x = ' + str(x.value))
    print('yf = ' + str(yf.value))
    print('zf = ' + str(zf.value))
    print('yh = ' + str(yh.value))
    print('zh = ' + str(zh.value))

optimal value = 760.0
x = [20.]
yf = [8.]
zf = [2.]
yh = [0.]
zh = [10.]


### Stock liquidation problem

In [9]:
# data
D = np.array([
    [100,20,30,36],
    [100,25,34,39], 
    [100,30,43,42],
    [100,35,47,45],
    [100,40,49,51],
    [100,45,53,55],
    [100,50,60,63],
    [100,55,62,64],
    [100,60,64,66],
    [100,65,66,70]
])
numShares = D[:,0]
purchasePrice = D[:,1]
currentPrice = D[:,2]
futurePrice = D[:,3]
N = len(numShares)
taxRate = 0.3
costRate = 0.01

In [10]:
x = cp.Variable(N)

In [11]:
tax = taxRate*sum([(currentPrice[k]-purchasePrice[k])*x[k] for k in range(N)])
saleProceeds = (1-costRate)*sum([currentPrice[k]*x[k] for k in range(N)]) - tax

constraints = [saleProceeds >= 30000, x >= 0, x <= numShares]
obj = cp.Maximize(sum([futurePrice[k]*(numShares[k]-x[k]) for k in range(N)]))

In [12]:
prob = cp.Problem(obj,constraints)
prob.solve(solver=cp.GLPK)
if (prob.status == 'optimal'):
    print('optimal value = ' + str(prob.value))
    print('x = ' + str(x.value))

optimal value = 20893.70880766926
x = [ 0.00000000e+00  0.00000000e+00  1.00000000e+02  1.00000000e+02
 -5.95608842e-14  6.37507490e+01 -1.42108547e-14  1.00000000e+02
  1.00000000e+02  1.00000000e+02]


### Refinery planning problem

In [ ]:
gasPrice = 12
gasGrade = 9
oilPrice = 5
oilGrade = 7
gasDemand = 2000
oilDemand = 600

In [ ]:
# Data
D = np.array([
    [0.3,0.5,0.8,3.40],
    [0.4,0.2,0.4,3.00],
    [0.1,0.3,0.2,2.60],
])
gradeOutput = D[:,range(3)]
price = D[:,3]
crackingPrice = np.array([1,1.5])

In [ ]:
x = cp.Variable(3) # crude processed through each of the methods
yg = cp.Variable(3) # crude converted to gas
yh = cp.Variable(3) # crude converted to heating oil
z = cp.Variable(2) # grade 6 -> grade 8, grade 8-> grade 10 conversion

In [ ]:
constraints = []

# inflow = outflow for grade 6
constraints += [sum([x[m]*gradeOutput[m,0] for m in range(3)]) ==  yg[0] + yh[0] + z[0]]
    
# inflow = outflow for grade 8
constraints += [sum([x[m]*gradeOutput[m,1] for m in range(3)]) + z[0] ==  yg[1] + yh[1] + z[1]]

# inflow = outflow for grade 10
constraints += [sum([x[m]*gradeOutput[m,2] for m in range(3)]) + z[1] ==  yg[2] + yh[2]]

# minimum grade constraint on gas
constraints += [6*yg[0] + 8*yg[1] + 10*yg[2] >= gasGrade*(yg[0]+yg[1]+yg[2])]

# minimum grade constraint on heating oil
constraints += [6*yh[0] + 8*yh[1] + 10*yh[2] >= oilGrade*(yh[0]+yh[1]+yh[2])]

# gas demand constraint
constraints += [sum([yg[k] for k in range(3)]) <= gasDemand]

# oil demand constraint
constraints += [sum([yh[k] for k in range(3)]) <= oilDemand]

# non-negativity constraints
constraints += [x >= 0, yg >= 0, yh >= 0, z >= 0]


In [ ]:
refiningCost = sum([x[k]*price[k] for k in range(3)])
crackingCost = sum([z[k]*crackingPrice[k] for k in range(2)])
revenue = gasPrice*sum([yg[k] for k in range(3)]) + oilPrice*sum([yh[k] for k in range(3)])
profit = revenue - refiningCost - crackingCost
obj = cp.Maximize(profit)

In [ ]:
prob = cp.Problem(obj,constraints)
prob.solve(solver=cp.GLPK)
if (prob.status == 'optimal'):
    print('optimal value = ' + str(prob.value))
    print('x = ' + str(x.value))
    print('yg = ' + str(yg.value))
    print('yh = ' + str(yh.value))
    print('revenue = ' + str(revenue.value))
    print('refining cost = ' + str(refiningCost.value))
    print('cracking cost = ' + str(crackingCost.value))

optimal value = 21475.0
x = [1625.    0.    0.]
yg = [ 300.  400. 1300.]
yh = [187.5 412.5   0. ]
revenue = 27000.0
refining cost = 5524.999999999999
cracking cost = -1.7053025658242404e-13
